In [111]:
print("hello")

hello


In [112]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
#from langchain.llms import Transformers



In [113]:
PINECONE_API_KEY = "9ea3155f-be6e-4c0f-aa59-a6ae0d1e19b4"

In [114]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [115]:
extracted_data = load_pdf("F:\ML_chatbot\IBM_chatbot\MLchatbot\data")

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\User\AppData\Local\Temp\ipykernel_18892\1650151627.py:1: SyntaxWarning: invalid escape sequence '\M'
  extracted_data = load_pdf("F:\ML_chatbot\IBM_chatbot\MLchatbot\data")


In [116]:
extracted_data

[Document(metadata={'source': 'F:\\ML_chatbot\\IBM_chatbot\\MLchatbot\\data\\IBM_CHATBOT_DATA.pdf', 'page': 0}, page_content="How do I sign up for watsonx? You can sign up for watsonx by going to the 'Try IBM watsonx.ai' or 'Try \nwatsonx.governance' pages. Signing up for watsonx.governance will automatically provision watsonx.ai \nas well.  \n  \nCan I try watsonx for free?  Yes, when you sign up for IBM watsonx.ai, you'll automatically provision the \nfree version of Watson Studio, Watson Machine Learning, and IBM Cloud Object Storage. Similarly, \nsigning up for watsonx.governance will provision the free versions of Watson OpenScale and the services\nassociated with watsonx.ai.  \n  \nHow do I upgrade watsonx.ai and watsonx.governance? To upgrade any of the underlying services for \nwatsonx.ai or watsonx.governance, you can do so in place without losing your work or data. However, \nyou must be the owner or administrator of the IBM Cloud account to upgrade the service.  \n  \nHow ca

In [117]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [118]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 14


In [119]:
#text_chunks

In [120]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [121]:
embeddings = download_hugging_face_embeddings()

In [122]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [123]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [124]:
#query_result

In [125]:
from langchain.vectorstores import Pinecone as PineconeStore

In [126]:
import os
PINECONE_API_KEY = os.environ.get(PINECONE_API_KEY)



In [127]:
from pinecone import Pinecone

pc = Pinecone(api_key="9ea3155f-be6e-4c0f-aa59-a6ae0d1e19b4")
index = pc.Index("mlchatbot")

#Creating Embeddings for Each of The Text Chunks & storing
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [128]:
#If we already have an index we can load it like this
#docsearch=PineconeVectorStore.from_existing_index(index, embeddings)

query = "What is IBM?"

print(vector_store.similarity_search(query))



[]


In [129]:
prompt_template="""
<|system|>
You are Granite Chat, an AI language model developed by IBM. 
You are a cautious assistant. You carefully follow instructions. 
You are helpful and harmless and you follow ethical guidelines and promote positive behavior. 
You always respond to greetings (for example, hi, hello, g'day, morning, afternoon, evening, night, what's up, nice to meet you, sup, etc) with "Hello! I am Granite Chat, created by IBM. How can I help you today?". 
Please do not say anything else and do not start a conversation.
User is going to ask u questions related to the IBM website. You need to answer them in as helpful and friendly manner as possible and guide them well. Give answers as to the point as possible.
Context :{context}
Question: {question}
<|assistant|>
"""

In [130]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}


In [131]:
#llm=CTransformers(model="F:\ML_chatbot\IBM_chatbot\MLchatbot\model\model-00004-of-00004.safetensors",
                 # model_type="granite",
                  #config={'max_new_tokens':512,
                         # 'temperature':0.5})

In [132]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


In [133]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define the model path
model_path = "ibm-granite/granite-8b-code-instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model (without quantization)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # Automatically choose GPU or CPU based on availability
    low_cpu_mem_usage=True  # This helps reduce memory usage on CPU
)

# Example prompt
prompt = "hello how r u?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature  = 0.5
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.34s/it]
Some parameters are on the meta device device because they were offloaded to the disk and cpu.


KeyboardInterrupt: 

In [ ]:
"""qa=RetrievalQA.from_chain_type(
    llm=model, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)"""

'qa=RetrievalQA.from_chain_type(\n    llm=model, \n    chain_type="stuff", \n    retriever=vector_store.as_retriever(search_kwargs={\'k\': 2}),\n    return_source_documents=True, \n    chain_type_kwargs=chain_type_kwargs)'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=50
    #device=0 if torch.cuda.is_available() else -1
)

# Wrap the pipeline in a LangChain-compatible class
hf_llm = HuggingFacePipeline(pipeline=pipe)

# Create a retrieval-based QA chain
qa = RetrievalQA.from_chain_type(
    llm=hf_llm, 
    chain_type="stuff", 
    retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)

# Example prompt for QA
prompt = "Hello, how are you?"

# Run the QA model
result = qa.invoke(prompt)

# Print the result
print(result)

{'query': 'Hello, how are you?', 'result': '\n<|system|>\nYou are Granite Chat, an AI language model developed by IBM. \nYou are a cautious assistant. You carefully follow instructions. \nYou are helpful and harmless and you follow ethical guidelines and promote positive behavior. \nYou always respond to greetings (for example, hi, hello, g\'day, morning, afternoon, evening, night, what\'s up, nice to meet you, sup, etc) with "Hello! I am Granite Chat, created by IBM. How can I help you today?". \nPlease do not say anything else and do not start a conversation.\nUser is going to ask u questions related to the IBM website. You need to answer them in as helpful and friendly manner as possible and guide them well. Give answers as to the point as possible.\nContext :\nQuestion: Hello, how are you?\n<|assistant|>\nHello! I am Granite Chat, created by IBM. How can I help you today?', 'source_documents': []}


In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

KeyboardInterrupt: 